In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import os
import csv
import tqdm

# Scraping

### env variables

In [2]:
BASE_PATH = '../data/pcs-scraping'
RESULTS_PATH = '../data/pcs-scraping/results/rider'
RANKINGS_PATH = '../data/pcs-scraping/pcs-rankings/rider'
TEAMS_PATH = '../data/pcs-scraping/teams/rider'

### Scraping functions

In [5]:
def get_rider_names(n_pages):
    
    rider_names = []

    offsets = np.arange(0, 3001, 100)

    for offset in offsets[:n_pages]:

        url = f'https://www.procyclingstats.com/rankings.php?date=2022-01-12&nation=&age=&zage=&page=smallerorequal&team=&offset={offset}&continent=&teamlevel=&filter=Filter&p=me&s=uci-individual'
        res = requests.get(url)

        tables = pd.read_html(res.content)
        rider_names.append(tables[0])

        time.sleep(0.5)
    
    return rider_names

def normalize_rider_name(rider_name):
    
    surname = rider_name.split(" ")[-1].lower()
    name = "-".join(rider_name.split(" ")[:-1]).lower()
    full_name = surname + '-' + name
    
    return full_name

def clean_pcs_table_results(df_table):
    
    df_table.drop('Unnamed: 3', axis=1, inplace=True)
    df_table.drop('Unnamed: 8', axis=1, inplace=True)
    df_table.rename(columns={'Unnamed: 2': 'GC'}, inplace=True)
    
    return df_table

def clean_pcs_table_ranking(df_table):
    
    df_table.rename(columns={'Unnamed: 0': 'year'}, inplace=True)

    return df_table

def get_rider_stats(rider_name, years):

    pcs_ranking = []
    results = []

    for year in years:

        try:
            url = f'https://www.procyclingstats.com/rider/{rider_name}/{year}'
            res = requests.get(url)

            tables = pd.read_html(res.content)
            pcs_ranking.append(clean_pcs_table_ranking(tables[1]))
            results.append(clean_pcs_table_results(tables[0]))

            time.sleep(0.5)
            
        except Exception as e:
            pass

    df_results = [(year, x) for year, x in zip(years, results) if not x.empty]
    
    return df_results, pcs_ranking[0]

def init_dirs():

    if not os.path.isdir(BASE_PATH):
        os.makedirs(BASE_PATH)
        
    if not os.path.isdir(RESULTS_PATH):
        os.makedirs(RESULTS_PATH)
        
    if not os.path.isdir(RANKINGS_PATH):
        os.makedirs(RANKINGS_PATH)
        
    if not os.path.isdir(TEAMS_PATH):
        os.makedirs(TEAMS_PATH)

def save_data(rider_name, results, pcs_ranking):
        
    # check if rider already has results data
    if not os.path.isdir(os.path.join(RESULTS_PATH, rider_name)):
        os.mkdir(os.path.join(RESULTS_PATH, rider_name))
    
    # check if rider already has pcs-ranking data
    if not os.path.isdir(os.path.join(RANKINGS_PATH, rider_name)):
        os.mkdir(os.path.join(RANKINGS_PATH, rider_name))
        
    # save season results
    #[x[1].to_csv(f'../data/pcs-scraping/results/rider/{rider_name}/{x[0]}.csv', index=False) for x in results]
    [x[1].to_csv(os.path.join(RESULTS_PATH, rider_name , f'{x[0]}.csv'), index=False) for x in results]
    
    # save pcs_ranking
    #pcs_ranking.to_csv(f'../data/pcs-scraping/pcs-ranking/rider/{rider_name}/pcs_ranking.csv', index=False)
    pcs_ranking.to_csv(os.path.join(RANKINGS_PATH, rider_name, 'pcs_ranking.csv'), index=False)

### Initialize data directories

In [6]:
init_dirs()

### Get Rider Names

In [7]:
rider_names = get_rider_names(n_pages=5)

In [8]:
riders = []
[riders.extend(x['Rider']) for x in rider_names]

rider_names = [normalize_rider_name(x) for x in riders]

In [9]:
with open(os.path.join(BASE_PATH, 'rider_names.csv'), 'w') as f:
    wr = csv.writer(f, quoting=csv.QUOTE_ALL)
    wr.writerow(rider_names)

### Get Race Results and Rankings

- NEED TO SCRAP ALL POSSIBLE YEARS (BEFORE 2011) !!!

- RIDER NAMES NOT CORRECT FOR FAILURES

In [10]:
years = np.arange(2011, 2022)

for rider_name in tqdm.tqdm(rider_names[:10]):

    try:
        results, pcs_ranking = get_rider_stats(rider_name, years)
        save_data(rider_name, results, pcs_ranking)
    except:
        print(rider_name)
        pass

100%|██████████| 10/10 [01:43<00:00, 10.40s/it]


### Get Teams

In [11]:
def get_rider_teams(rider_name):
    
    time.sleep(0.5)
    
    url = f'https://www.procyclingstats.com/rider/{rider_name}'
    res = requests.get(url)
    
    soup = BeautifulSoup(res.content)
    
    possible_classes = ['list rdr-teams moblist moblist', 'list rdr-teams moblist', 'list rdr-teams moblist moblist ']
    ul = list(filter(None, [soup.find('ul', {'class': class_}) for class_ in possible_classes]))
    
    if ul:
        
        season = [x.find('div', {'class': 'season'}).text for x in ul[0].find_all('li')]
        team = [x.find('div', {'class': 'name'}).text for x in ul[0].find_all('li')]
        teams = pd.DataFrame({'season': season, 'team': team})
        
        return teams

    else:
        
        print(rider_name, 'No Teams scraped')

def save_data(rider_name, teams):
        
    # check if rider already has teams data
    if not os.path.isdir(os.path.join(TEAMS_PATH, rider_name)):
        os.mkdir(os.path.join(TEAMS_PATH, rider_name))
        
    # save teams data
    teams.to_csv(os.path.join(TEAMS_PATH, rider_name, 'teams.csv'), index=False)

In [12]:
rider_names = list(pd.read_csv(os.path.join(BASE_PATH, 'rider_names.csv')))

In [14]:
for rider_name in tqdm.tqdm(rider_names[:10]):
    
    try:
        teams = get_rider_teams(rider_name)
        save_data(rider_name, teams)
    except:
        print(rider_name)

100%|██████████| 10/10 [00:10<00:00,  1.01s/it]


- RIDER NAMES NOT CORRECT ON URL FOR FAILURES